# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [12]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "Phi4-mini"

In [13]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [17]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [18]:
# Let's just make sure the model is loaded

import subprocess

def run_ollama_pull(model):
    process = subprocess.Popen(
        ["ollama", "pull", model],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    for line in process.stdout:
        print('\r' + line.strip(), end='', flush=True)
    process.wait()
    print()  # Move to next line after completion

run_ollama_pull(MODEL)

Python(10678) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


success [K▕██████████████████▏  411 B                         nifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 


In [19]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative Artificial Intelligence (GenAI) refers to systems that can generate new content, such as text, images, and sounds. These technologies leverage deep learning models like GPT-3 or DALL-E for creating outputs based on input data they were trained on.

Several areas have seen the application of GenAI in business:

1. Content Creation: Generative AI tools help create articles, blog posts, videos scripts etc., reducing time spent manually generating these materials.
2. Personalization and Recommendation Systems: Companies can use generative models to generate personalized product recommendations or marketing messages based on customer's preferences/past behavior data.

3. Customer Service Chatbots: These GenAI-powered chat bots are capable of providing accurate responses by understanding customers queries better than human agents, saving costs in the process.
4. Data Analysis and Visualization Tools : With Generative AI tools businesses can generate visualizations like graphs/char

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [20]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative Artificial Intelligence (GenAI) refers to a subset of machine learning where models can generate new data samples that resemble given input. These technologies are increasingly being applied across various industries and markets, offering innovative solutions for businesses facing complex challenges.

1. Content Creation: GenAI is transforming the way content creation works in media companies like publishing houses or advertising agencies by generating original articles, reports, stories, videos etc., thus saving significant resources on cost while boosting productivity.
2. Product Design: Companies can use generative AI tools to create unique and innovative designs for products ranging from fashion items such as clothes with personalized prints/embellishments/sizes/designs to furniture pieces like chairs or tables having customized features according to specific customer preferences/patterns
3. Marketing Automation : Marketers are leveraging GenAI techniques in automating t

## Alternative approach - using OpenAI python library to connect to Ollama

In [21]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative Artificial Intelligence (GenAI) represents a significant evolution in how artificial intelligence can be applied across various industries. As businesses look to leverage GenAI, its transformative potential becomes apparent:


1. **Content Creation and Marketing**: Companies like BuzzFeed Inc. are already utilizing generative models such as ChatGPT for producing engaging social media posts that cater directly to consumer interests.


2. **Personalized Experiences in Retailing & E-Commerce**: Amazon uses advanced AI algorithms not just at the point-of-sale but across their entire website, improving customer experience through personalized shopping suggestions based on search queries and browsing behavior without requiring explicit user input.



4. **Healthcare Solutions for Drug Discovery**: GenAI is being used to analyze complex biological data more efficiently in hopes of discovering new drugs or repurposing existing ones.


5. **Enhanced Customer Service with Chatbots & V

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [22]:
!ollama pull deepseek-r1:1.5b

Python(10997) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 269 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 476 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.3 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.0 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.0 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 3.4 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 4.2 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 4.9 MB/1

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import ollama



class SeleniumWebsite:
    def __init__(self, url, wait_time=5):
        self.url = url
        self.wait_time = wait_time
        self.html = None

    def fetch(self):
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
        driver = webdriver.Chrome(options=options)
        try:
            driver.get(self.url)
            time.sleep(self.wait_time)  # Wait for JS to load
            self.html = driver.page_source
        finally:
            driver.quit()
        return self.html

    def summarize(self):
        if self.html is None:
            self.fetch()
        system_prompt = (
            "You are a highly capable assistant that analyzes the contents of a website and provides a short summary, "
            "ignoring text that might be navigation related. Respond in markdown."
        )
        user_prompt = (
            f"You are looking at a website at {self.url}\n"
            "The contents of this website is as follows; please provide a short summary of this website in markdown. "
            "If it includes news or announcements, then summarize these too.\n\n"
            f"{self.html}"
        )
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        response = ollama.chat(model=MODEL, messages=messages)

        return response['message']['content']

def display_summary(url):
    site = SeleniumWebsite(url)
    summary = site.summarize()
    from IPython.display import Markdown, display
    display(Markdown(summary))

In [29]:
display_summary("https://openai.com")

Python(12111) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(12140) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Looks like there is a script tag with a source attribute including an inline JavaScript function. I need to make sure the site can execute this code correctly without being blocked by security policies such as X-Frame options or Content Security Policy (CSP). In your case, if it's for personal use and you have control over both sites involved in any cross-origin frame embedding (`https://example.com` here is a placeholder), here's what to do:

1. Add `X-Frame Options: ALLOW-FROM https://example.com`.
2. Allow this site (from which the script originates) as well.
3. Ensure that your own site's CSP settings allow for scripts of type 'script' from any origin with at least the same or more lenient policies compared to what's set by `https://cdn-cgi`.

For a third-party service like Google's, you'll have limited control over their security headers; if you need help making changes there (such as submitting an issue through support channels), please get in touch directly via [Google's script embedding documentation](https://developers.google.com/).

If both sites can be adjusted to allow such embeds without compromising on the overall web application's other CSP directives, this should work fine.

For further customization or specific security concerns that might require more complex configuration (e.g., handling cross-origin scripts with high privileges), please contact a network/security professional. Do not run untrusted code from embedded third-party sources unless you fully understand its impact and have ensured it is safe to do so on the intended system.